<a href="https://colab.research.google.com/github/ProfAI/tf00/blob/master/6%20-%20Generatori%20e%20Images%20Augmentation/images_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Images Augmentation
Per addestrare una rete neurale, specialmente una molto profonda, è necessario avere a disposizione una grande quantità di dati. Se il dataset che abbiamo è limitato, una soluzione potrebbe consistere nel generare nuovi dati sintetici partendo da quelli di cui già disponiamo. Nell'ambito della computer vision questa pratica è chiamata Images Aumentation, in questo notebook vedremo come applicarla utilizzando i generatori di tf.keras.

## Scarichiamo il Dataset
In questo notebook proveremo a riconoscere foto di minori, a questo scopo utilizzeremo un dataset di immagini di volti di persone, classificate come minorenni e maggiorenni

In [1]:
!wget https://profession.ai/datasets/adult_or_minor.zip
!unzip adult_or_minor.zip


--2020-07-03 15:09:50--  https://profession.ai/datasets/adult_or_minor.zip
Resolving profession.ai (profession.ai)... 13.224.198.88, 13.224.198.125, 13.224.198.128, ...
Connecting to profession.ai (profession.ai)|13.224.198.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23453058 (22M) [application/zip]
Saving to: ‘adult_or_minor.zip’

adult_or_minor.zip  100%[===================>]  22.37M   146MB/s    in 0.2s    

2020-07-03 15:09:50 (146 MB/s) - ‘adult_or_minor.zip’ saved [23453058/23453058]

Archive:  adult_or_minor.zip
   creating: adult_or_minor/
  inflating: adult_or_minor/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/adult_or_minor/
  inflating: __MACOSX/adult_or_minor/._.DS_Store  
   creating: adult_or_minor/adults/
  inflating: adult_or_minor/adults/63.jpg  
  inflating: adult_or_minor/adults/77.jpg  
  inflating: adult_or_minor/adults/604.jpg  
  inflating: adult_or_minor/adults/176.jpg  
  inflating: adult_or_minor/adults/610.jpg  
  

## Importiamo i Moduli

In [4]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, Flatten, Dense

from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Definiamo le Costanti

In [7]:
DATASET_DIR = "adult_or_minor/"
BATCH_SIZE = 64

TOT_IMAGES = 1000 
IMG_SIZE = (228, 228)
VALIDATION_SPLIT = 0.2

## Usiamo il Generatori per Generare nuove Immagini
Per generare nuove immagini dobbiamo applicare delle trasformazioni alle immagini di cui già disponiamo, come rotazioni, translazioni o zoom. Per definire diverse trasformazioni da applicare possiamo utilizzare i parametri della classe *ImageDataGenerator*. [Qui](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) puoi trovare la lista di tutti i parametri e di tutte le possibili trasformazioni.

In [8]:
datagen_train = ImageDataGenerator(validation_split=VALIDATION_SPLIT, 
                            rescale=1./255,
                            shear_range=0.2, 
                            zoom_range=0.2, 
                            horizontal_flip=True)

Adesso possiamo creare i generatori di immagini per addestramento e validazione.

In [9]:
train_generator = datagen_train.flow_from_directory(
        DATASET_DIR,
        target_size=IMG_SIZE, 
        batch_size=BATCH_SIZE,
        class_mode='binary',
        subset='training')

valid_generator = datagen_train.flow_from_directory(
        DATASET_DIR,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        subset='validation')


print(train_generator.class_indices)

Found 764 images belonging to 2 classes.
Found 190 images belonging to 2 classes.
{'adults': 0, 'minors': 1}


## Addestriamo la Rete Neurale Convoluzionale sul Generatore
Utilizziamo la stessa architettura che abbiamo definito per l'esempio dei generatori.

In [11]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))
model.add(MaxPooling2D(pool_size=3, strides=3))
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=3, strides=3))
model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=3, strides=3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 228, 228, 32)      416       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 76, 76, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 76, 76, 32)        4128      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 64)        8256      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
dense (Dense)                (None, 8, 8, 128)         8

Il processo di addestramento è esattamente ugualea quanto già fatto per i generatori.

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                              min_delta=0.01,
                              patience=3,
                              restore_best_weights=True)

model.fit(
        train_generator,
        steps_per_epoch=int(TOT_IMAGES*(1.-VALIDATION_SPLIT) // BATCH_SIZE),
        validation_data=valid_generator, 
        validation_steps=int(TOT_IMAGES*VALIDATION_SPLIT // BATCH_SIZE),
        epochs=50)

Epoch 1/100
12/12 [==============================] - 11s 887ms/step - loss: 0.6985 - accuracy: 0.5105 - val_loss: 0.6942 - val_accuracy: 0.5053
Epoch 2/100
12/12 [==============================] - 11s 907ms/step - loss: 0.6960 - accuracy: 0.4895 - val_loss: 0.6925 - val_accuracy: 0.5211
Epoch 3/100
12/12 [==============================] - 11s 902ms/step - loss: 0.6934 - accuracy: 0.5026 - val_loss: 0.6925 - val_accuracy: 0.5053
Epoch 4/100
12/12 [==============================] - 11s 888ms/step - loss: 0.6935 - accuracy: 0.4843 - val_loss: 0.6914 - val_accuracy: 0.6263
Epoch 5/100
12/12 [==============================] - 10s 873ms/step - loss: 0.6912 - accuracy: 0.5327 - val_loss: 0.6869 - val_accuracy: 0.6211
Epoch 6/100
12/12 [==============================] - 10s 867ms/step - loss: 0.6808 - accuracy: 0.5707 - val_loss: 0.6786 - val_accuracy: 0.5684
Epoch 7/100
12/12 [==============================] - 10s 857ms/step - loss: 0.6720 - accuracy: 0.5982 - val_loss: 0.6649 - val_accuracy:

# Testiamo la Rete sul Generatore
Per testare sul generatore possiamo sempre usare il metodo *evaluate*.

In [ ]:
metrics_train = model.evaluate(train_generator)
metrics_test = model.evaluate(test_generator)

print("Train Accuracy = %.4f - Train Loss = %.4f" % (metrics_train[1], metrics_train[0]))
print("Test Accuracy = %.4f - Test Loss = %.4f" % (metrics_test[1], metrics_test[0]))

20/20 [==============================] - 18s 911ms/step - loss: 2.2509 - accuracy: 0.4144
Train Accuracy = 0.9725 - Train Loss = 0.1707
Test Accuracy = 0.4144 - Test Loss = 2.2509
